In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
# GMMA+ADX Daily Scheduler — Documentation

This doc explains how to use, configure, and operate the **daily scheduler** built on your GMMA+ADX framework. It runs once **after market close (post 15:30 IST)**, generates BUY/SELL recommendations with reasons, and maintains state in CSVs.

---

## 1) What the scheduler does

* **Executes** any previously scheduled orders whose `entry_date == today`:

  * `PENDING_BUY` → fills at **today’s Open** → becomes `OPEN`
  * `PENDING_SELL` → fills at **today’s Open** → position removed; PnL logged
* **Scans exits** for all `OPEN` positions on **today’s daily bar**:

  * Priority: **stop**, **target**, then **GMMA break**
  * Schedules exit as `PENDING_SELL` for **next business day** (fills at next Open)
* **Scans new BUYs** for **today**:

  * Signal: **ADX cross above level** AND **all GMMA(short) > all GMMA(long)**
  * Filters: within **52-week high threshold**, **VOLAr** ranking
  * Respects `max_concurrent_positions`
  * Schedules `PENDING_BUY` for **next business day** (fills at next Open)
* **Writes**:

  * `outputs/positions.csv` (persistent portfolio state)
  * `outputs/trades_log.csv` (executed BUY/SELL fills with fees, PnL)
  * `outputs/recommendations_YYYYMMDD.csv` (human-readable actions for next session)

---

## 2) Quick start

1. Save the script as `gmma_adx_scheduler.py`.
2. Create a universe:

   * Either set `CFG.static_symbols = ['RELIANCE.NS', 'TCS.NS', ...]`
   * Or keep `CFG.static_symbols_path = "nifty500.txt"` (one symbol per line, **use .NS**)
3. (Optional) Place NSE holiday dates in `config/nse_holidays_2025.csv` (one `YYYY-MM-DD` per line) and set:

   ```python
   CFG.holiday_csv = "config/nse_holidays_2025.csv"
   ```
4. Run once after **15:30 IST**:

   ```bash
   python gmma_adx_scheduler.py
   ```
5. Check outputs in the `outputs/` folder.

---

## 3) Files & folders

```
project/
├── gmma_adx_scheduler.py
├── nifty500.txt                 # your universe (optional)
├── config/
│   └── nse_holidays_2025.csv    # optional holiday list
├── cache/                       # auto-created price cache (per symbol .parquet)
└── outputs/
    ├── positions.csv
    ├── trades_log.csv
    └── recommendations_YYYYMMDD.csv
```

---

## 4) CSV schemas

### 4.1 `outputs/positions.csv`

Tracks current & pending positions.

| column      | type      | note                                                          |
| ----------- | --------- | ------------------------------------------------------------- |
| ticker      | str       | e.g., `RELIANCE.NS`                                           |
| status      | str       | `OPEN`, `PENDING_BUY`, `PENDING_SELL`                         |
| entry_date  | date      | **Trading date** the order will/was filled (daily index date) |
| entry_price | float     | Filled price (Open of `entry_date`) for buys; NaN for pending |
| shares      | int       | Rounded down based on allocation                              |
| stop_px     | float     | `entry_price * (1 - stop_loss_pct)`                           |
| tgt_px      | float     | `entry_price * (1 + target_pct)`                              |
| buy_fee     | float     | Fees charged on BUY fill                                      |
| notes       | str       | Reason strings & audit info                                   |
| created_at  | timestamp | IST timestamp row created                                     |
| updated_at  | timestamp | IST timestamp last updated                                    |

**Lifecycle**
`PENDING_BUY` → (fill next open) → `OPEN` → (exit signal) → `PENDING_SELL` → (fill next open) → removed.

---

### 4.2 `outputs/trades_log.csv`

Executed BUY/SELL fills (after they actually happen).

| column   | type  | note                                                          |
| -------- | ----- | ------------------------------------------------------------- |
| ticker   | str   | Symbol                                                        |
| side     | str   | `BUY` or `SELL`                                               |
| date     | date  | Trading date filled (Open)                                    |
| price    | float | Fill price                                                    |
| shares   | int   | Shares                                                        |
| reason   | str   | Why this fill happened                                        |
| fees_inr | float | Fees for this leg                                             |
| pnl_inr  | float | Net PnL (for SELL) = (sell - buy)*shares - buy_fee - sell_fee |

---

### 4.3 `outputs/recommendations_YYYYMMDD.csv`

Human-readable daily plan generated after you run the scheduler.

| column          | type | example                                                              |
| --------------- | ---- | -------------------------------------------------------------------- |
| date            | date | 2025-10-20                                                           |
| for_action_date | date | 2025-10-21                                                           |
| type            | str  | `BUY (next open)` / `SELL (next open)`                               |
| ticker          | str  | `INFY.NS`                                                            |
| reason          | str  | `ADX↑>30 & GMMA(short)>GMMA(long); 52w%=91.3%; VOLAr=0.88; ADX=34.6` |

---

## 5) Configuration guide (key fields)

Inside `Config`:

* **Data**

  * `start_date`, `end_date=None` (uses today)
  * `static_symbols` or `static_symbols_path`
* **Signals**

  * `gmma_short=(3,5,8,10,12,15)`
  * `gmma_long=(30,35,40,45,50,60)`
  * `adx_len=14`, `adx_cross_level=30.0`
  * `within_pct_of_52w_high=0.50` (keep stocks ≥ 50% of 52W high)
* **Risk**

  * `stop_loss_pct=0.05`, `target_pct=0.10`
* **Portfolio**

  * `max_concurrent_positions=5`
  * `top_k_daily=300` (ranked candidates cap)
  * `per_trade_inr=100_000.0` (allocation per new entry)
  * `apply_fees=True` (approx Groww-like)
* **Calendar**

  * `force_today_after_close=True` (use **today** if run after 15:30 IST)
  * `market_close_hhmm="15:30"`
  * `holiday_csv=None` (optional NSE holidays CSV)
* **Liquidity (optional)**

  * `enable_basic_liquidity=False`
  * `min_price_inr=50.0`, `min_avg_vol_20d=50_000.0`

---

## 6) How the trading calendar works

* When you run after **15:30 IST** and `force_today_after_close=True`, the scheduler sets:

  * `signal_day = today (IST)` if it’s a weekday and not a holiday.
* It computes `next_td` as the **next business day** (skips weekends + holidays).
* This avoids `next=None` even if the benchmark index doesn’t yet include “tomorrow”.

**Tip:** If you must run before close, set `force_today_after_close=False` so it uses the last completed benchmark day instead.

---

## 7) Daily run workflow (example)

1. **You run** on Mon **2025-10-20** at 17:00 IST.
2. The script:

   * Fills any orders scheduled for 2025-10-20 at **Open[2025-10-20]**
   * Scans **exits** for `OPEN` positions using the 2025-10-20 bar → schedules `PENDING_SELL` for **2025-10-21**
   * Scans **new BUYs** for 2025-10-20 → schedules `PENDING_BUY` for **2025-10-21**
   * Writes `recommendations_20251020.csv`

On Tue **2025-10-21** after close, run again: it will **fill** all `PENDING_*` dated 2025-10-21 at **Open[2025-10-21]**.

---

## 8) Starting from scratch or migrating

* **Fresh start:** Ensure `outputs/positions.csv` does **not** exist; the script will create it.
* **Migrating existing positions:** Create `outputs/positions.csv` with your open holdings:

  ```
  ticker,status,entry_date,entry_price,shares,stop_px,tgt_px,buy_fee,notes,created_at,updated_at
  INFY.NS,OPEN,2025-10-10,1650.00,60,1567.50,1815.00,18.50,seed import,2025-10-20T16:05:00,2025-10-20T16:05:00
  ```

  Missing columns will be auto-added as NaN; fill what you know.

---

## 9) Scheduling (automation)

* **Linux/macOS (cron):**

  ```
  # Run at 16:05 IST on weekdays
  35 10 * * 1-5 /usr/bin/python3 /path/gmma_adx_scheduler.py >> /path/cron.log 2>&1
  ```

  (10:35 UTC ≈ 16:05 IST; adjust for DST/host TZ.)
* **Windows (Task Scheduler):**

  * Trigger: Daily at 16:05
  * Action: `python.exe` with arguments `C:\path\gmma_adx_scheduler.py`

---

## 10) Common issues & fixes

* **`next=None` in logs**
  Use the provided script version. It computes next business day independent of benchmark rows.
* **No recommendations generated**

  * The bar may be incomplete (ran before close). Either run after **15:30 IST** or set `force_today_after_close=False`.
  * Your thresholds may be tight (`adx_cross_level`, `within_pct_of_52w_high`). Relax to test.
* **Symbols missing data**
  Ensure `.NS` suffix. Check `cache/` was written—if a symbol fails from YF it’s skipped.
* **Timezone confusion**
  Daily bars have **naive** dates; we use **IST clock** for deciding “today” and “next” but keep trading dates naive for indexing consistency.
* **Fees look off**
  The fee model is an approximation. Tune `calc_fees` as per your broker contract.

---

## 11) Customization ideas

* **Sizing**: Replace `per_trade_inr` with your **MVO sizing + cash cap** (you already have that in your backtester). The place to plug in is where `PENDING_BUY` fills and where BUYs are **scheduled**.
* **Additional filters**: Add 200-EMA trend filter, minimum ADV, sector whitelist, etc., inside `day_buy_signals`.
* **Different exits**: Add time-based exit, trailing stop, ADX weaken, etc., in `eval_exit_signals`.
* **Notifications**: Tail a simple Telegram webhook after writing `recommendations_*.csv`.

---

## 12) FAQ

**Q: Can I run multiple times a day?**
A: Yes; only the latest run after close matters. Running pre-close with `force_today_after_close=True` will still try to use “today”, which might lead to acting on an incomplete bar—avoid that; or set it to `False` if you must run earlier.

**Q: How do I limit the universe to NIFTY50?**
A: Put the 50 symbols in a text file and set `CFG.static_symbols_path` to that file.

**Q: What about yfinance limits?**
A: The script caches each symbol to `cache/*.parquet`. The scheduler only re-downloads if the last cached date < `end_date`.

---

## 13) Sanity checklist before going live

* [ ] Universe file has **.NS** suffixes
* [ ] Holiday CSV configured (optional but recommended)
* [ ] `max_concurrent_positions` and `per_trade_inr` align with your risk
* [ ] Run once manually after close; inspect `recommendations_*.csv`
* [ ] Review `positions.csv` lifecycle after first 2–3 days

---

Need this tailored to your **MVO sizing** and **deploy-cash cap** like in your backtester? Say the word—I’ll slot that logic into the scheduler while preserving the same CSV state machine.

"""

import os, json, math, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd

try:
    import yfinance as yf
except Exception:
    pass

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# LOGGING
# =========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
log = logging.getLogger("gmma_adx_daily_scheduler")

IST = "Asia/Kolkata"

# =========================
# CONFIG
# =========================
@dataclass
class Config:
    # Data
    start_date: str = "2015-01-01"
    end_date: Optional[str] = None  # None => up to today (IST)
    static_symbols: Optional[List[str]] = None
    static_symbols_path: Optional[str] = None
    cache_dir: str = "cache"
    out_dir: str   = "outputs"
    plot: bool     = False  # Scheduler doesn't plot

    # GMMA
    gmma_short: Tuple[int,...] = (3,5,8,10,12,15)
    gmma_long:  Tuple[int,...] = (30,35,40,45,50,60)

    # ADX
    adx_len: int = 14
    adx_cross_level: float = 30.0

    # Risk management
    stop_loss_pct: float = 0.05
    target_pct: float    = 0.10

    # Portfolio policy (scheduler)
    max_concurrent_positions: int = 5
    top_k_daily: int = 300
    per_trade_inr: float = 100_000.0   # fixed allocation per new entry
    apply_fees: bool = True

    # Entry/Exit execution model
    entry_on_next_open: bool = True
    exit_on_next_open: bool = True

    # Ranking & filters
    benchmark_try: Tuple[str,...] = ("^CNX500","^CRSLDX","^NSE500","^NIFTY500","^BSE500","^NSEI")
    volar_lookback: int = 252
    filter_52w_window: int = 252
    within_pct_of_52w_high: float = 0.50

    # Liquidity (optional)
    enable_basic_liquidity: bool = False
    min_price_inr: float = 50.0
    min_avg_vol_20d: float = 50_000.0

    # Exchange calendar controls (NEW)
    force_today_after_close: bool = True
    market_close_hhmm: str = "15:30"        # IST close
    holiday_csv: Optional[str] = None       # CSV of YYYY-MM-DD (NSE holidays)

CFG = Config()

# =========================
# FEES (Groww-style ballpark)
# =========================
APPLY_FEES = True
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not APPLY_FEES:
        return 0.0
    BROKER_PCT = 0.001; BROKER_MIN = 5.0; BROKER_CAP = 20.0
    STT_PCT = 0.001
    STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297
    SEBI_PCT = 0.000001
    IPFT_PCT = 0.000001
    GST_PCT = 0.18
    DP_SELL = 20.0 if turnover_sell >= 100 else 0.0
    def _broker(turnover):
        if turnover <= 0: return 0.0
        fee = turnover * BROKER_PCT
        return max(BROKER_MIN, min(fee, BROKER_CAP))
    br_buy = _broker(turnover_buy); br_sell = _broker(turnover_sell)
    stt = STT_PCT * (turnover_buy + turnover_sell)
    stamp = STAMP_BUY_PCT * turnover_buy
    misc = (EXCH_PCT + SEBI_PCT + IPFT_PCT) * (turnover_buy + turnover_sell)
    gst = GST_PCT * (br_buy + br_sell + misc + DP_SELL)
    return float(br_buy + br_sell + stt + stamp + misc + DP_SELL + gst)

# =========================
# Helpers
# =========================
def ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def today_ist_date_str() -> str:
    return pd.Timestamp.now(tz=IST).strftime("%Y-%m-%d")

def load_static_symbols(static_symbols: Optional[List[str]], static_symbols_path: Optional[str]) -> List[str]:
    syms: List[str] = []
    if static_symbols and len(static_symbols) > 0:
        syms = list(static_symbols)
    elif static_symbols_path and os.path.exists(static_symbols_path):
        with open(static_symbols_path, "r") as f:
            syms = [line.strip() for line in f if line.strip()]
    else:
        raise ValueError("Provide CFG.static_symbols=[...] (.NS) or CFG.static_symbols_path file.")
    out = []
    for s in syms:
        s = s.strip().upper()
        if not s.endswith(".NS"):
            s = f"{s}.NS"
        out.append(s)
    uniq = []
    seen = set()
    for s in out:
        if s not in seen:
            uniq.append(s); seen.add(s)
    return uniq

def fetch_prices(tickers: List[str], start: str, end: Optional[str], cache_dir: str) -> Dict[str, pd.DataFrame]:
    ensure_dirs(cache_dir)
    data = {}
    end = end or today_ist_date_str()
    for ticker in tickers:
        cache_path = os.path.join(cache_dir, f"{ticker.replace('^', '_')}.parquet")
        use_cache = False
        if os.path.exists(cache_path):
            try:
                df = pd.read_parquet(cache_path)
                if len(df) and pd.to_datetime(df.index[-1]).strftime("%Y-%m-%d") >= end:
                    data[ticker] = df; use_cache = True
            except Exception:
                pass
        if use_cache:
            continue
        try:
            df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False, multi_level_index=False)
            if df is None or df.empty:
                continue
            df = df.rename(columns=str.title)[['Open','High','Low','Close','Volume']].dropna()
            df.index.name = "date"
            df.to_parquet(cache_path)
            data[ticker] = df
        except Exception:
            continue
    return data

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

def _true_range(high: pd.Series, low: pd.Series, prev_close: pd.Series) -> pd.Series:
    return pd.concat([(high-low).abs(), (high-prev_close).abs(), (low-prev_close).abs()], axis=1).max(axis=1)

def adx(high: pd.Series, low: pd.Series, close: pd.Series, length: int = 14) -> pd.Series:
    prev_high, prev_low, prev_close = high.shift(1), low.shift(1), close.shift(1)
    up_move, down_move = high - prev_high, prev_low - low
    plus_dm  = up_move.where((up_move > down_move) & (up_move > 0), 0.0)
    minus_dm = down_move.where((down_move > up_move) & (down_move > 0), 0.0)
    tr = _true_range(high, low, prev_close)
    alpha = 1.0/length
    atr = tr.ewm(alpha=alpha, adjust=False, min_periods=length).mean()
    plus_di  = 100 * (plus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    minus_di = 100 * (minus_dm.ewm(alpha=alpha, adjust=False, min_periods=length).mean() / atr)
    dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0, np.nan)
    return dx.ewm(alpha=alpha, adjust=False, min_periods=length).mean()

def gmma_frames(df: pd.DataFrame, short_p: Tuple[int,...], long_p: Tuple[int,...]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    s = pd.DataFrame({f"S{p}": ema(df["Close"], p) for p in short_p}, index=df.index)
    l = pd.DataFrame({f"L{p}": ema(df["Close"], p) for p in long_p}, index=df.index)
    return s, l

def compute_indicators(df: pd.DataFrame, cfg: Config) -> pd.DataFrame:
    out = df.copy()
    s, l = gmma_frames(out, cfg.gmma_short, cfg.gmma_long)
    out = out.join(s).join(l)
    out["min_short"] = s.min(axis=1); out["max_short"] = s.max(axis=1)
    out["min_long"]  = l.min(axis=1); out["max_long"]  = l.max(axis=1)
    out["adx"] = adx(out["High"], out["Low"], out["Close"], cfg.adx_len)
    out["adx_prev"] = out["adx"].shift(1)
    out["avg_vol_20"] = out["Volume"].rolling(20).mean()
    out["high_52w"]   = out["Close"].rolling(cfg.filter_52w_window).max()
    return out.dropna()

def basic_liquidity_ok(row: pd.Series, cfg: Config) -> bool:
    if not cfg.enable_basic_liquidity: return True
    if row["Close"] < cfg.min_price_inr: return False
    if row["avg_vol_20"] < cfg.min_avg_vol_20d: return False
    return True

# =========================
# Benchmark & VOLAr
# =========================
def pick_benchmark(benchmarks: Tuple[str,...], start: str, end: Optional[str], cache_dir: str) -> Tuple[str, pd.DataFrame]:
    for t in benchmarks:
        data = fetch_prices([t], start, end, cache_dir)
        df = data.get(t)
        if df is not None and not df.empty:
            log.info("Using benchmark: %s", t)
            return t, df
    idx = pd.date_range(start=start, end=end or today_ist_date_str(), freq="B")
    df = pd.DataFrame({"Close": np.ones(len(idx))}, index=idx)
    log.warning("No benchmark found; using synthetic flat series.")
    return "SYNTH_BENCH", df

def compute_volar_scores(end_dt: pd.Timestamp, tickers: List[str], data_map: Dict[str,pd.DataFrame], bench_df: pd.DataFrame, lookback: int) -> Dict[str, float]:
    scores = {}
    bser = bench_df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
    for t in tickers:
        df = data_map.get(t)
        if df is None or df.empty:
            scores[t] = 0.0; continue
        r = df["Close"].loc[:end_dt].pct_change().dropna().iloc[-lookback:]
        common = pd.concat([r, bser], axis=1, keys=["s","b"]).dropna()
        if common.shape[0] < max(20, int(0.4*lookback)):
            scores[t] = 0.0; continue
        excess = common["s"] - common["b"]
        vol = common["s"].std(ddof=0)
        scores[t] = 0.0 if vol <= 1e-8 else float((excess.mean() / vol) * math.sqrt(252.0))
    return scores

# =========================
# Positions & Trades storage
# =========================
POS_COLS = [
    "ticker","status","entry_date","entry_price","shares","stop_px","tgt_px",
    "buy_fee","notes","created_at","updated_at"
]
TRD_COLS = [
    "ticker","side","date","price","shares","reason","fees_inr","pnl_inr"
]
def positions_path(cfg: Config) -> str:
    return os.path.join(cfg.out_dir, "positions.csv")
def trades_log_path(cfg: Config) -> str:
    return os.path.join(cfg.out_dir, "trades_log.csv")
def recos_path(cfg: Config, dt: pd.Timestamp) -> str:
    return os.path.join(cfg.out_dir, f"recommendations_{pd.Timestamp(dt).strftime('%Y%m%d')}.csv")

def load_positions(cfg: Config) -> pd.DataFrame:
    p = positions_path(cfg)
    if os.path.exists(p):
        df = pd.read_csv(p)
        for c in POS_COLS:
            if c not in df.columns:
                df[c] = np.nan
        # keep trading dates naive (align with yfinance daily index)
        df["entry_date"] = pd.to_datetime(df["entry_date"])
        df["created_at"] = pd.to_datetime(df["created_at"])
        df["updated_at"] = pd.to_datetime(df["updated_at"])
        return df[POS_COLS].copy()
    return pd.DataFrame(columns=POS_COLS)

def save_positions(df: pd.DataFrame, cfg: Config):
    df = df.copy()
    df.to_csv(positions_path(cfg), index=False)

def append_trade_log(rows: List[dict], cfg: Config):
    path = trades_log_path(cfg)
    if os.path.exists(path):
        old = pd.read_csv(path)
        new = pd.DataFrame(rows, columns=TRD_COLS)
        out = pd.concat([old, new], ignore_index=True)
    else:
        out = pd.DataFrame(rows, columns=TRD_COLS)
    out.to_csv(path, index=False)

# =========================
# Trading day helpers (robust)
# =========================
from datetime import time as _time

def _load_holidays(path: Optional[str]) -> set:
    if not path or not os.path.exists(path):
        return set()
    s = pd.read_csv(path, header=None)[0].astype(str).str.strip()
    return set(pd.to_datetime(s).dt.strftime("%Y-%m-%d").tolist())

def _is_business_day(d: pd.Timestamp, holidays: set) -> bool:
    # 'd' is naive date (midnight). Weekend check + holiday check.
    if d.weekday() >= 5:
        return False
    return d.strftime("%Y-%m-%d") not in holidays

def _next_business_day(d: pd.Timestamp, holidays: set) -> pd.Timestamp:
    x = d + pd.Timedelta(days=1)
    while not _is_business_day(x, holidays):
        x += pd.Timedelta(days=1)
    return x

def resolve_signal_and_next_days(cfg: Config, bench_df: pd.DataFrame) -> Tuple[pd.Timestamp, pd.Timestamp]:
    """
    Decide 'signal_day' (the day whose signals we evaluate) and 'next_td' (the day on which
    pending orders will fill at the open), without requiring the benchmark to contain 'next_td'.
    """
    now_ist = pd.Timestamp.now(tz=IST)
    hh, mm = map(int, getattr(cfg, "market_close_hhmm", "15:30").split(":"))
    market_closed = now_ist.time() >= _time(hh, mm)

    today_naive = pd.to_datetime(now_ist.strftime("%Y-%m-%d"))  # naive date
    holidays = _load_holidays(getattr(cfg, "holiday_csv", None))

    if cfg.force_today_after_close and market_closed and _is_business_day(today_naive, holidays):
        signal_day = today_naive
    else:
        # fallback to last available benchmark day (e.g., before close or non-business day)
        signal_day = pd.to_datetime(bench_df.index[-1])

    next_td = _next_business_day(signal_day, holidays)
    return signal_day, next_td

# =========================
# Signal logic (one-day scan)
# =========================
def day_buy_signals(dt: pd.Timestamp, data_map: Dict[str,pd.DataFrame], cfg: Config, bench_df: pd.DataFrame) -> pd.DataFrame:
    recs = []
    for tkr, df in data_map.items():
        if df is None or df.empty or dt not in df.index:
            continue
        d = compute_indicators(df, cfg)
        if dt not in d.index:
            continue
        row = d.loc[dt]
        adx_cross = (row["adx_prev"] <= cfg.adx_cross_level) and (row["adx"] > cfg.adx_cross_level)
        long_ok = (row["min_short"] > row["max_long"])
        if adx_cross and long_ok and basic_liquidity_ok(row, cfg):
            close = float(row["Close"])
            high_52w = float(row["high_52w"])
            pct_52w = (close / high_52w) if (high_52w and high_52w>0) else np.nan
            recs.append({
                "ticker": tkr,
                "date": dt,
                "close": close,
                "high_52w": high_52w,
                "pct_of_52w": pct_52w,
                "adx": float(row["adx"]),
                "min_short": float(row["min_short"]),
                "max_long": float(row["max_long"]),
                "signal_reason": f"ADX↑>{cfg.adx_cross_level:.0f} & GMMA(short)>GMMA(long)"
            })
    out = pd.DataFrame(recs)
    if out.empty:
        return out

    # 52w filter
    out = out[out["pct_of_52w"] >= cfg.within_pct_of_52w_high].copy()
    if out.empty:
        return out

    # VOLAr ranking
    tickers = out["ticker"].tolist()
    volar = compute_volar_scores(dt, tickers, data_map, bench_df, cfg.volar_lookback)
    out["volar"] = out["ticker"].map(volar)
    out = out.sort_values(["volar","adx","pct_of_52w"], ascending=[False, False, False]).reset_index(drop=True)
    return out

def eval_exit_signals(dt: pd.Timestamp, pos_df: pd.DataFrame, data_map: Dict[str,pd.DataFrame], cfg: Config) -> pd.DataFrame:
    """
    For each OPEN position, check if today's bar generates an exit signal.
    Exit is scheduled for next trading day (next open fill).
    Priorities: stop/target hit (today's H/L), else GMMA break.
    """
    rows = []
    for _, pos in pos_df[pos_df["status"]=="OPEN"].iterrows():
        tkr = pos["ticker"]
        df = data_map.get(tkr)
        if df is None or df.empty or dt not in df.index:
            continue
        ind = compute_indicators(df, cfg)
        if dt not in ind.index:
            continue
        row = ind.loc[dt]
        today = df.loc[dt]

        entry_px = float(pos["entry_price"])
        stop_px  = float(pos["stop_px"])
        tgt_px   = float(pos["tgt_px"])

        hit = None
        if (today["Low"] <= stop_px) and (today["High"] >= tgt_px):
            hit = ("SELL", "target")
        elif today["Low"] <= stop_px:
            hit = ("SELL", "stop")
        elif today["High"] >= tgt_px:
            hit = ("SELL", "target")
        elif not (row["min_short"] > row["max_long"]):
            hit = ("SELL", "gmma_break")

        if hit is not None:
            rows.append({
                "ticker": tkr,
                "side": hit[0],
                "signal_date": dt,
                "reason": hit[1],
                "note": f"Exit scheduled next open: {hit[1]} | entry={entry_px:.2f}, stop={stop_px:.2f}, tgt={tgt_px:.2f}"
            })
    return pd.DataFrame(rows)

# =========================
# Fees helpers
# =========================
def fee_for_buy(turnover: float) -> float: return calc_fees(turnover, 0.0)
def fee_for_sell(turnover: float) -> float: return calc_fees(0.0, turnover)

# =========================
# Scheduler main
# =========================
def scheduler_run(cfg: Config):
    ensure_dirs(cfg.cache_dir, cfg.out_dir)
    global APPLY_FEES
    APPLY_FEES = bool(cfg.apply_fees)

    # Universe & data
    symbols = load_static_symbols(cfg.static_symbols, cfg.static_symbols_path)
    sym_set = set(symbols)

    # Pull prices
    data_map = fetch_prices(symbols, cfg.start_date, cfg.end_date, cfg.cache_dir)

    # Benchmark for calendar reference
    bench_tkr, bench_df = pick_benchmark(cfg.benchmark_try, cfg.start_date, cfg.end_date, cfg.cache_dir)
    if bench_df is None or bench_df.empty:
        raise RuntimeError("No benchmark data to determine trading days.")

    # Resolve robust dates (FIXED)
    today_td, next_td = resolve_signal_and_next_days(cfg, bench_df)
    log.info("Trading calendar: today=%s | next=%s", today_td.date(), next_td.date())

    # Load or init positions
    pos = load_positions(cfg)
    now_ts = pd.Timestamp.now(tz=IST)

    # Drop symbols no longer in universe (optional)
    if not pos.empty:
        pos = pos[pos["ticker"].isin(sym_set)].copy()

    # 1) EXECUTE pending orders whose date == today_td
    executed_trades = []

    # PENDING_BUY -> fill at today's Open
    pend_buy = pos[(pos["status"]=="PENDING_BUY") & (pos["entry_date"]==today_td)].copy()
    for i, r in pend_buy.iterrows():
        tkr = r["ticker"]; df = data_map.get(tkr)
        if df is None or df.empty or today_td not in df.index:
            continue
        px = float(df.loc[today_td, "Open"])
        shares = int(max(1, math.floor(float(cfg.per_trade_inr) / px)))
        buy_turn = shares * px
        fee = fee_for_buy(buy_turn)
        stop_px = px * (1 - cfg.stop_loss_pct)
        tgt_px  = px * (1 + cfg.target_pct)
        pos.loc[i, ["status","entry_price","shares","stop_px","tgt_px","buy_fee","updated_at"]] = \
            ["OPEN", px, shares, stop_px, tgt_px, fee, now_ts]
        executed_trades.append({
            "ticker": tkr, "side":"BUY", "date": today_td, "price": px, "shares": shares,
            "reason": "filled pending BUY at next open", "fees_inr": fee, "pnl_inr": 0.0
        })
        log.info("FILLED BUY  %-12s @ %.2f sh=%d (fee %.2f) -> OPEN", tkr, px, shares, fee)

    # PENDING_SELL -> fill at today's Open; remove from positions
    pend_sell = pos[(pos["status"]=="PENDING_SELL") & (pos["entry_date"]==today_td)].copy()
    rows_to_drop = []
    for i, r in pend_sell.iterrows():
        tkr = r["ticker"]; df = data_map.get(tkr)
        if df is None or df.empty or today_td not in df.index:
            continue
        px = float(df.loc[today_td, "Open"])
        shares = int(r["shares"])
        sell_turn = shares * px
        fee = fee_for_sell(sell_turn)
        buy_fee = float(r.get("buy_fee", 0.0))
        pnl = (px - float(r["entry_price"])) * shares - buy_fee - fee
        executed_trades.append({
            "ticker": tkr, "side":"SELL", "date": today_td, "price": px, "shares": shares,
            "reason": r.get("notes","filled pending SELL at next open"), "fees_inr": fee, "pnl_inr": pnl
        })
        rows_to_drop.append(i)
        log.info("FILLED SELL %-12s @ %.2f sh=%d (fee %.2f) -> CLOSED | net PnL=%.2f", tkr, px, shares, fee, pnl)
    if rows_to_drop:
        pos = pos.drop(index=rows_to_drop).reset_index(drop=True)

    if executed_trades:
        append_trade_log(executed_trades, cfg)

    # 2) EVALUATE exits for all OPEN positions on today_td -> schedule PENDING_SELL for next_td
    exit_signals = eval_exit_signals(today_td, pos, data_map, cfg)
    planned_exits = []
    if (exit_signals is not None) and (not exit_signals.empty):
        for _, sig in exit_signals.iterrows():
            tkr = sig["ticker"]
            idx = pos[(pos["ticker"]==tkr) & (pos["status"]=="OPEN")].index
            if len(idx)==0:
                continue
            i = idx[0]
            pos.loc[i, ["status","entry_date","updated_at","notes"]] = \
                ["PENDING_SELL", next_td, now_ts, sig["note"]]
            planned_exits.append({
                "type": "SELL",
                "ticker": tkr,
                "for_date": next_td,
                "reason": sig["reason"],
                "note": sig["note"]
            })
            log.info("SCHEDULE SELL %-12s on %s | %s", tkr, str(next_td.date()), sig["note"])

    # 3) SCAN new BUY signals today -> schedule PENDING_BUY for next_td (respect slots)
    open_count = (pos["status"]=="OPEN").sum()
    pending_buy_count = (pos["status"]=="PENDING_BUY").sum()
    slots_left = max(0, int(cfg.max_concurrent_positions) - int(open_count) - int(pending_buy_count))
    planned_buys = []
    if slots_left > 0:
        buys = day_buy_signals(today_td, data_map, cfg, bench_df)
        if (buys is not None) and (not buys.empty):
            # exclude already held/pending
            held = set(pos[pos["status"].isin(["OPEN","PENDING_BUY"])]["ticker"].tolist())
            buys = buys[~buys["ticker"].isin(held)].reset_index(drop=True)
            if not buys.empty:
                select = buys.head(min(cfg.top_k_daily, slots_left)).copy()
                for _, r in select.iterrows():
                    tkr = r["ticker"]
                    reason = f"{r['signal_reason']}; 52w%={r['pct_of_52w']:.1%}; VOLAr={r['volar']:.2f}; ADX={r['adx']:.1f}"
                    pos = pd.concat([pos, pd.DataFrame([{
                        "ticker": tkr,
                        "status": "PENDING_BUY",
                        "entry_date": next_td,  # will fill next open
                        "entry_price": np.nan,
                        "shares": np.nan,
                        "stop_px": np.nan,
                        "tgt_px": np.nan,
                        "buy_fee": np.nan,
                        "notes": reason,
                        "created_at": now_ts,
                        "updated_at": now_ts
                    }])], ignore_index=True)
                    planned_buys.append({
                        "type":"BUY",
                        "ticker": tkr,
                        "for_date": next_td,
                        "reason": reason
                    })
                    log.info("SCHEDULE BUY  %-12s on %s | %s", tkr, str(next_td.date()), reason)

    # 4) Persist artifacts
    ensure_dirs(cfg.out_dir)
    save_positions(pos, cfg)

    # Human-friendly daily recommendations CSV
    reco_rows = []
    for b in planned_buys:
        reco_rows.append({
            "date": today_td.date(),
            "for_action_date": b["for_date"].date(),
            "type": "BUY (next open)",
            "ticker": b["ticker"],
            "reason": b["reason"]
        })
    for e in planned_exits:
        reco_rows.append({
            "date": today_td.date(),
            "for_action_date": e["for_date"].date(),
            "type": "SELL (next open)",
            "ticker": e["ticker"],
            "reason": f"{e['reason']}: {e['note']}"
        })
    recos = pd.DataFrame(reco_rows, columns=["date","for_action_date","type","ticker","reason"])
    recos.to_csv(recos_path(cfg, today_td), index=False)

    # 5) Console summary
    log.info("=== SUMMARY (%s) ===", str(today_td.date()))
    log.info("Open positions: %d | Pending BUY: %d | Pending SELL: %d",
             int((pos['status']=='OPEN').sum()), int((pos['status']=='PENDING_BUY').sum()), int((pos['status']=='PENDING_SELL').sum()))
    if not recos.empty:
        log.info("Recommendations saved -> %s", recos_path(cfg, today_td))
    else:
        log.info("No new BUY/SELL scheduled for next session.")

def main():
    # === Configure your universe here ===
    # Example:
    # CFG.static_symbols = ['RELIANCE.NS','TCS.NS','HDFCBANK.NS','INFY.NS','ICICIBANK.NS']
    CFG.static_symbols_path = "nifty500.txt"  # or comment the line above and use static_symbols

    # Optional: NSE holidays CSV (YYYY-MM-DD one per line)
    # CFG.holiday_csv = "config/nse_holidays_2025.csv"

    scheduler_run(CFG)

if __name__ == "__main__":
    main()


<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_89682/819214419.py:4: SyntaxWarning: invalid escape sequence '\p'
  """
2025-10-28 22:50:13 | ERROR | HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ^CNX500"}}}
2025-10-28 22:50:13 | ERROR | 
1 Failed download:
2025-10-28 22:50:13 | ERROR | ['^CNX500']: YFTzMissingError('possibly delisted; no timezone found')
2025-10-28 22:50:13 | INFO | Using benchmark: ^CRSLDX
2025-10-28 22:50:13 | INFO | Trading calendar: today=2025-10-28 | next=2025-10-29
2025-10-28 22:50:14 | INFO | === SUMMARY (2025-10-28) ===
2025-10-28 22:50:14 | INFO | Open positions: 0 | Pending BUY: 0 | Pending SELL: 0
2025-10-28 22:50:14 | INFO | No new BUY/SELL scheduled for next session.
